cell 1

In [110]:
# !pip install langchain openai pinecone-client python-dotenv

from pinecone import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()


True

CELL 2




In [111]:
pc = Pinecone(api_key='ea33957d-5f26-4206-a51a-4babdcdf6291')
# openai api key is stored in .env file

CELL 3

In [112]:
history=[
"1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
"2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
"3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
"4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
"5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
"6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
"7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
"8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
"9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
"10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
"11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
"12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
"13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
"14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
"15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
"16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
"17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
"18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
"19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
"20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

embeddings=OpenAIEmbeddings(api_key=os.environ['openai_api'])

query="Do you think it will help me stay fit?"

template="Write an response for  {query} where chat history are {history_data}"


CELL 4

In [113]:
def add_embeddings_to_pinecone(history, index_name='sahil'):

    ids=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
    vector_doc=[]
   
    for i in  history:
        vector_doc.append(embeddings.embed_query(i))
        
    index = pc.Index("sahil")
    pc.describe_index(name=index_name)
    index.upsert(vectors=zip(ids,vector_doc))
    
add_embeddings_to_pinecone(history)


CELL 5

In [114]:
def retrieve_relevant_history(query,index_name='sahil'):
    index=pc.Index(index_name)
    matching_ans=index.query(vector=embeddings.embed_query(query),top_k=3,include_values=False)
    matching_ids = [match['id'] for match in matching_ans['matches']]
    similarity=list(map(lambda x:history[int(x)],matching_ids))
    return  similarity
    

CELL 6

In [115]:
def prepare_prompt(test_prompt):
    prompt=PromptTemplate(input_variables=['query','history_data'],template=test_prompt)
    matching_answer=retrieve_relevant_history(query)
    return prompt,matching_answer
    

CELL 7

In [116]:
def test_final_prompt(question):

    llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.9,max_tokens=225,api_key=os.environ["openai_api"])
    prompt,matching_ref=prepare_prompt(template)
    gen=LLMChain(prompt=prompt,llm=llm)
    final_response=gen.run({"query": question, "history_data": matching_ref})
    print(f"Final Test Prompt: {prompt}")
    print(f"Context Referred: {matching_ref}")
    print(f"Final Test Prompt Response: {final_response}")
    return final_response


In [117]:
test_final_prompt(query)

Final Test Prompt: input_variables=['history_data', 'query'] template='Write an response for  {query} where chat history are {history_data}'
Context Referred: ['8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.', "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.", "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument."]
Final Test Prompt Response: I think incorporating running into your fitness routine, along with exploring hobbies like painting, hiking, or learning a new instrument, will definitely help you stay fit. Running is a great form of cardio that can help improve your cardiovascular health and burn calories. Additionally, engaging in hobbies you enjoy will not only keep you active but also contribute to your overall well-being. Remember, consistenc

'I think incorporating running into your fitness routine, along with exploring hobbies like painting, hiking, or learning a new instrument, will definitely help you stay fit. Running is a great form of cardio that can help improve your cardiovascular health and burn calories. Additionally, engaging in hobbies you enjoy will not only keep you active but also contribute to your overall well-being. Remember, consistency is key, so finding activities that you genuinely enjoy will make it easier for you to stick to your fitness goals. Good luck on your fitness journey!'